In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import warnings
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import pickle
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
from statsmodels.tsa.statespace.sarimax import SARIMAX
import copy
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
warnings.filterwarnings(action='ignore')
from epftoolbox.evaluation import MAE, sMAPE

C:\Users\UX431\anaconda3\lib\site-packages\statsmodels\compat\pandas.py:61: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


# DNN

In [2]:
df = pd.read_csv(r"Data\EPFToolbox\FR.csv")

In [3]:
df

,Date,Prices,Generation forecast,System load forecast
0,2011-01-09 00:00:00,32.542,63065.0,63000.0
1,2011-01-09 01:00:00,21.549,62715.0,58800.0
2,2011-01-09 02:00:00,15.711,61952.0,58500.0
3,2011-01-09 03:00:00,10.583,59262.0,54300.0
4,2011-01-09 04:00:00,10.324,56883.0,51900.0
...,...,...,...,...
52411,2016-12-31 19:00:00,69.720,70329.0,73957.0
52412,2016-12-31 20:00:00,57.100,69121.0,72544.0
52413,2016-12-31 21:00:00,54.490,66647.0,69451.0
52414,2016-12-31 22:00:00,54.950,65886.0,67823.0


In [4]:
df['Date'] = pd.to_datetime(df.Date)

In [5]:
df.index = df.Date

In [6]:
df = df.drop('Date', axis = 1)

In [7]:
df.columns = ['Price', 'Exogenous 1', 'Exogenous 2']

In [8]:
df_train = df[df.index < dt.datetime(2015, 1, 1)]
df_test = df[df.index >= dt.datetime(2015, 1, 1)]

In [9]:
forecast = pd.DataFrame(index=df_test.index[::24], columns=['h' + str(k) for k in range(24)])
real_values = df_test.loc[:, ['Price']].values.reshape(-1, 24)
real_values = pd.DataFrame(real_values, index=forecast.index, columns=forecast.columns)

In [10]:
forecast_dates = forecast.index

In [12]:
from epftoolbox.models import hyperparameter_optimizer

In [13]:
df.to_csv("Data/EPFToolbox/FR_dnn.csv")

In [14]:
# Number of layers in DNN
nlayers = 2

# Market under study. If it not one of the standard ones, the file name
# has to be provided, where the file has to be a csv file
dataset = "FR_dnn"

# Number of years (a year is 364 days) in the test dataset.
years_test = 2

# Optional parameters for selecting the test dataset, if either of them is not provided, 
# the test dataset is built using the years_test parameter. They should either be one of
# the date formats existing in python or a string with the following format
# "%d/%m/%Y %H:%M"
begin_test_date = "01/01/2015 00:00"
end_test_date = "31/12/2016 23:00"

# Boolean that selects whether the validation and training datasets are shuffled
shuffle_train = 1

# Boolean that selects whether a data augmentation technique for DNNs is used
data_augmentation = 0

# Boolean that selects whether we start a new hyperparameter optimization or we restart an existing one
new_hyperopt = 1

# Number of years used in the training dataset for recalibration
calibration_window = 4

# Unique identifier to read the trials file of hyperparameter optimization
experiment_id = 1

# Number of iterations for hyperparameter optimization
max_evals = 100

path_datasets_folder = "./Data/EPFToolbox/"
path_hyperparameters_folder = "./experimental_files/"

# Check documentation of the hyperparameter_optimizer for each of the function parameters
# In this example, we optimize a model for the PJM market.
# We consider two directories, one for storing the datasets and the other one for the experimental files.
# We start a hyperparameter optimization from scratch. We employ 1500 iterations in hyperopt,
# 2 years of test data, a DNN with 2 hidden layers, a calibration window of 4 years,
# we avoid data augmentation,  and we provide an experiment_id equal to 1
hyperparameter_optimizer(path_datasets_folder=path_datasets_folder, 
                         path_hyperparameters_folder=path_hyperparameters_folder, 
                         new_hyperopt=new_hyperopt, max_evals=max_evals, nlayers=nlayers, dataset=dataset, 
                         years_test=years_test, calibration_window=calibration_window, 
                         shuffle_train=shuffle_train, data_augmentation=0, experiment_id=experiment_id,
                         begin_test_date=begin_test_date, end_test_date=end_test_date)

Test datasets: 2015-01-01 00:00:00 - 2016-12-31 23:00:00


Tested 1/100 iterations.
Best MAE - Validation Dataset
  MAE: 7.8 | sMAPE: 17.02 %

Best MAE - Test Dataset
  MAE: 6.9 | sMAPE: 19.26 %


Tested 2/100 iterations.
Best MAE - Validation Dataset
  MAE: 7.8 | sMAPE: 17.02 %

Best MAE - Test Dataset
  MAE: 6.9 | sMAPE: 19.26 %


Tested 3/100 iterations.
Best MAE - Validation Dataset
  MAE: 7.8 | sMAPE: 17.02 %

Best MAE - Test Dataset
  MAE: 6.9 | sMAPE: 19.26 %


Tested 4/100 iterations.
Best MAE - Validation Dataset
  MAE: 7.8 | sMAPE: 17.02 %

Best MAE - Test Dataset
  MAE: 6.9 | sMAPE: 19.26 %


Tested 5/100 iterations.
Best MAE - Validation Dataset
  MAE: 7.8 | sMAPE: 17.02 %

Best MAE - Test Dataset
  MAE: 6.9 | sMAPE: 19.26 %


Tested 6/100 iterations.
Best MAE - Validation Dataset
  MAE: 7.8 | sMAPE: 17.02 %

Best MAE - Test Dataset
  MAE: 6.9 | sMAPE: 19.26 %


Tested 7/100 iterations.
Best MAE - Validation Dataset
  MAE: 7.8 | sMAPE: 17.02 %

Best MAE - Test Dataset
  MAE



Tested 60/100 iterations.
Best MAE - Validation Dataset
  MAE: 5.7 | sMAPE: 12.50 %

Best MAE - Test Dataset
  MAE: 4.6 | sMAPE: 13.11 %


Tested 61/100 iterations.
Best MAE - Validation Dataset
  MAE: 5.7 | sMAPE: 12.50 %

Best MAE - Test Dataset
  MAE: 4.6 | sMAPE: 13.11 %


Tested 62/100 iterations.
Best MAE - Validation Dataset
  MAE: 5.7 | sMAPE: 12.50 %

Best MAE - Test Dataset
  MAE: 4.6 | sMAPE: 13.11 %


Tested 63/100 iterations.
Best MAE - Validation Dataset
  MAE: 5.7 | sMAPE: 12.50 %

Best MAE - Test Dataset
  MAE: 4.6 | sMAPE: 13.11 %


Tested 64/100 iterations.
Best MAE - Validation Dataset
  MAE: 5.7 | sMAPE: 12.50 %

Best MAE - Test Dataset
  MAE: 4.6 | sMAPE: 13.11 %


Tested 65/100 iterations.
Best MAE - Validation Dataset
  MAE: 5.7 | sMAPE: 12.50 %

Best MAE - Test Dataset
  MAE: 4.6 | sMAPE: 13.11 %


Tested 66/100 iterations.
Best MAE - Validation Dataset
  MAE: 5.7 | sMAPE: 12.50 %

Best MAE - Test Dataset
  MAE: 4.6 | sMAPE: 13.11 %


Tested 67/100 iterations.

In [16]:
"""
Example for using the DNN model for forecasting prices with daily recalibration
"""

# Author: Jesus Lago

# License: AGPL-3.0 License

import pandas as pd
import numpy as np
import argparse
import os

from epftoolbox.data import read_data
from epftoolbox.evaluation import MAE, sMAPE
from epftoolbox.models import DNN


path_datasets_folder = "./Data/EPFToolbox/"
path_hyperparameters_folder = "./experimental_files/"
path_recalibration_folder = os.path.join('.', 'Resultados')
path_hyperparameter_folder = os.path.join('.', 'experimental_files')

dataset = "FR_dnn"

# Number of years (a year is 364 days) in the test dataset.
years_test = 2

# Optional parameters for selecting the test dataset, if either of them is not provided, 
# the test dataset is built using the years_test parameter. They should either be one of
# the date formats existing in python or a string with the following format
# "%d/%m/%Y %H:%M"
begin_test_date = "01/01/2015 00:00"
end_test_date = "31/12/2016 23:00"

# Defining train and testing data
df_train, df_test = read_data(dataset=dataset, years_test=years_test, path=path_datasets_folder,
                              begin_test_date=begin_test_date, end_test_date=end_test_date)

# Defining unique name to save the forecast
forecast_file_name = 'dnn_FR.csv'

forecast_file_path = os.path.join(path_recalibration_folder, forecast_file_name)

# # Defining empty forecast array and the real values to be predicted in a more friendly format
forecast = pd.DataFrame(index=df_test.index[::24], columns=['h' + str(k) for k in range(24)])
real_values = df_test.loc[:, ['Price']].values.reshape(-1, 24)
real_values = pd.DataFrame(real_values, index=forecast.index, columns=forecast.columns)

# # If we are not starting a new recalibration but re-starting an old one, we import the
# # existing files and print metrics 
# if not new_recalibration:
#     # Import existinf forecasting file
#     forecast = pd.read_csv(forecast_file_path, index_col=0)
#     forecast.index = pd.to_datetime(forecast.index)

#     # Reading dates to still be forecasted by checking NaN values
#     forecast_dates = forecast[forecast.isna().any(axis=1)].index

#     # If all the dates to be forecasted have already been forecast, we print information
#     # and exit the script
#     if len(forecast_dates) == 0:

#         mae = np.mean(MAE(forecast.values.squeeze(), real_values.values))
#         smape = np.mean(sMAPE(forecast.values.squeeze(), real_values.values)) * 100
#         print('{} - sMAPE: {:.2f}%  |  MAE: {:.3f}'.format('Final metrics', smape, mae))
    
# else:
#     forecast_dates = forecast.index

forecast_dates = forecast.index

nlayers = 2
shuffle_train = 1
data_augmentation = 0
calibration_window = 4
experiment_id = 1

model = DNN(
    experiment_id=experiment_id, path_hyperparameter_folder=path_hyperparameter_folder, nlayers=nlayers, 
    dataset=dataset, years_test=years_test, shuffle_train=shuffle_train, data_augmentation=data_augmentation,
    calibration_window=calibration_window)


# For loop over the recalibration dates
for date in forecast_dates:

    # For simulation purposes, we assume that the available data is
    # the data up to current date where the prices of current date are not known
    data_available = pd.concat([df_train, df_test.loc[:date + pd.Timedelta(hours=23), :]], axis=0)

    # We extract real prices for current date and set them to NaN in the dataframe of available data
    data_available.loc[date:date + pd.Timedelta(hours=23), 'Price'] = np.NaN

    # Recalibrating the model with the most up-to-date available data and making a prediction
    # for the next day
    Yp = model.recalibrate_and_forecast_next_day(df=data_available, next_day_date=date)

    # Saving the current prediction
    forecast.loc[date, :] = Yp

    # Computing metrics up-to-current-date
    mae = np.mean(MAE(forecast.loc[:date].values.squeeze(), real_values.loc[:date].values)) 
    smape = np.mean(sMAPE(forecast.loc[:date].values.squeeze(), real_values.loc[:date].values)) * 100

    # Pringint information
    print('{} - sMAPE: {:.2f}%  |  MAE: {:.3f}'.format(str(date)[:10], smape, mae))

#     # Saving forecast
#     forecast.to_csv(forecast_file_path)

Test datasets: 2015-01-01 00:00:00 - 2016-12-31 23:00:00
2015-01-01 - sMAPE: 11.32%  |  MAE: 4.434
2015-01-02 - sMAPE: 10.16%  |  MAE: 3.915
2015-01-03 - sMAPE: 8.69%  |  MAE: 3.396
2015-01-04 - sMAPE: 9.45%  |  MAE: 3.419
2015-01-05 - sMAPE: 10.15%  |  MAE: 3.961
2015-01-06 - sMAPE: 10.06%  |  MAE: 4.069
2015-01-07 - sMAPE: 9.47%  |  MAE: 3.887
2015-01-08 - sMAPE: 8.99%  |  MAE: 3.682
2015-01-09 - sMAPE: 9.24%  |  MAE: 3.660
2015-01-10 - sMAPE: 10.86%  |  MAE: 3.808
2015-01-11 - sMAPE: 12.18%  |  MAE: 3.945
2015-01-12 - sMAPE: 12.24%  |  MAE: 4.017
2015-01-13 - sMAPE: 11.94%  |  MAE: 3.917
2015-01-14 - sMAPE: 12.09%  |  MAE: 3.983
2015-01-15 - sMAPE: 12.01%  |  MAE: 3.938
2015-01-16 - sMAPE: 12.36%  |  MAE: 4.055
2015-01-17 - sMAPE: 12.12%  |  MAE: 3.980
2015-01-18 - sMAPE: 11.83%  |  MAE: 3.901
2015-01-19 - sMAPE: 11.52%  |  MAE: 3.844
2015-01-20 - sMAPE: 11.24%  |  MAE: 3.792
2015-01-21 - sMAPE: 11.04%  |  MAE: 3.764
2015-01-22 - sMAPE: 10.92%  |  MAE: 3.781
2015-01-23 - sMAPE: 10.7

2015-07-16 - sMAPE: 11.71%  |  MAE: 4.072
2015-07-17 - sMAPE: 11.69%  |  MAE: 4.068
2015-07-18 - sMAPE: 11.68%  |  MAE: 4.064
2015-07-19 - sMAPE: 11.67%  |  MAE: 4.062
2015-07-20 - sMAPE: 11.65%  |  MAE: 4.054
2015-07-21 - sMAPE: 11.65%  |  MAE: 4.063
2015-07-22 - sMAPE: 11.62%  |  MAE: 4.058
2015-07-23 - sMAPE: 11.59%  |  MAE: 4.052
2015-07-24 - sMAPE: 11.56%  |  MAE: 4.043
2015-07-25 - sMAPE: 11.78%  |  MAE: 4.102
2015-07-26 - sMAPE: 11.88%  |  MAE: 4.111
2015-07-27 - sMAPE: 11.91%  |  MAE: 4.116
2015-07-28 - sMAPE: 11.93%  |  MAE: 4.115
2015-07-29 - sMAPE: 11.93%  |  MAE: 4.113
2015-07-30 - sMAPE: 11.93%  |  MAE: 4.113
2015-07-31 - sMAPE: 11.91%  |  MAE: 4.105
2015-08-01 - sMAPE: 11.91%  |  MAE: 4.099
2015-08-02 - sMAPE: 11.90%  |  MAE: 4.087
2015-08-03 - sMAPE: 11.88%  |  MAE: 4.078
2015-08-04 - sMAPE: 11.91%  |  MAE: 4.086
2015-08-05 - sMAPE: 11.88%  |  MAE: 4.078
2015-08-06 - sMAPE: 11.85%  |  MAE: 4.068
2015-08-07 - sMAPE: 11.83%  |  MAE: 4.059
2015-08-08 - sMAPE: 11.82%  |  MAE

2016-01-28 - sMAPE: 11.85%  |  MAE: 4.087
2016-01-29 - sMAPE: 11.84%  |  MAE: 4.083
2016-01-30 - sMAPE: 11.87%  |  MAE: 4.085
2016-01-31 - sMAPE: 11.89%  |  MAE: 4.082
2016-02-01 - sMAPE: 11.93%  |  MAE: 4.090
2016-02-02 - sMAPE: 11.95%  |  MAE: 4.088
2016-02-03 - sMAPE: 11.96%  |  MAE: 4.089
2016-02-04 - sMAPE: 11.96%  |  MAE: 4.089
2016-02-05 - sMAPE: 11.96%  |  MAE: 4.086
2016-02-06 - sMAPE: 11.97%  |  MAE: 4.084
2016-02-07 - sMAPE: 11.99%  |  MAE: 4.081
2016-02-08 - sMAPE: 12.05%  |  MAE: 4.088
2016-02-09 - sMAPE: 12.05%  |  MAE: 4.085
2016-02-10 - sMAPE: 12.04%  |  MAE: 4.080
2016-02-11 - sMAPE: 12.02%  |  MAE: 4.074
2016-02-12 - sMAPE: 12.04%  |  MAE: 4.077
2016-02-13 - sMAPE: 12.03%  |  MAE: 4.073
2016-02-14 - sMAPE: 12.06%  |  MAE: 4.071
2016-02-15 - sMAPE: 12.06%  |  MAE: 4.070
2016-02-16 - sMAPE: 12.09%  |  MAE: 4.076
2016-02-17 - sMAPE: 12.07%  |  MAE: 4.072
2016-02-18 - sMAPE: 12.07%  |  MAE: 4.072
2016-02-19 - sMAPE: 12.07%  |  MAE: 4.070
2016-02-20 - sMAPE: 12.07%  |  MAE

2016-08-11 - sMAPE: 11.96%  |  MAE: 3.675
2016-08-12 - sMAPE: 11.95%  |  MAE: 3.673
2016-08-13 - sMAPE: 11.96%  |  MAE: 3.674
2016-08-14 - sMAPE: 11.98%  |  MAE: 3.675
2016-08-15 - sMAPE: 12.00%  |  MAE: 3.677
2016-08-16 - sMAPE: 11.99%  |  MAE: 3.675
2016-08-17 - sMAPE: 11.99%  |  MAE: 3.675
2016-08-18 - sMAPE: 11.97%  |  MAE: 3.670
2016-08-19 - sMAPE: 11.97%  |  MAE: 3.670
2016-08-20 - sMAPE: 11.96%  |  MAE: 3.667
2016-08-21 - sMAPE: 12.00%  |  MAE: 3.669
2016-08-22 - sMAPE: 11.99%  |  MAE: 3.666
2016-08-23 - sMAPE: 11.98%  |  MAE: 3.663
2016-08-24 - sMAPE: 11.97%  |  MAE: 3.660
2016-08-25 - sMAPE: 11.96%  |  MAE: 3.658
2016-08-26 - sMAPE: 11.95%  |  MAE: 3.657
2016-08-27 - sMAPE: 11.95%  |  MAE: 3.657
2016-08-28 - sMAPE: 11.96%  |  MAE: 3.657
2016-08-29 - sMAPE: 11.95%  |  MAE: 3.657
2016-08-30 - sMAPE: 11.94%  |  MAE: 3.654
2016-08-31 - sMAPE: 11.93%  |  MAE: 3.653
2016-09-01 - sMAPE: 11.92%  |  MAE: 3.651
2016-09-02 - sMAPE: 11.91%  |  MAE: 3.649
2016-09-03 - sMAPE: 11.91%  |  MAE

In [17]:
forecast

,h0,h1,h2,h3,h4,h5,h6,h7,h8,h9,...,h14,h15,h16,h17,h18,h19,h20,h21,h22,h23
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-01,45.726528,42.415802,38.358463,33.200146,30.107626,31.186077,30.55687,32.579582,37.157436,41.784981,...,38.325237,36.32658,36.653606,43.589748,50.636066,52.508476,49.046734,45.343426,48.735897,46.066322
2015-01-02,44.463581,41.097034,37.481064,32.294174,30.256001,32.596783,34.894211,39.109669,41.803558,45.32225,...,42.139927,40.004581,41.047504,47.771404,55.908627,57.633984,53.411068,47.909054,50.287643,47.569584
2015-01-03,43.692715,40.804489,37.495617,33.003471,31.241756,32.973576,34.847588,37.72831,40.996449,44.577782,...,39.977142,38.450935,39.084671,45.890667,52.439419,52.814236,47.499695,42.98526,45.938198,44.231106
2015-01-04,38.215508,34.640316,30.833439,26.418869,23.946123,24.230165,24.832575,26.69989,30.879374,36.229187,...,34.15139,32.280571,33.644745,40.739594,47.333988,48.912693,43.445885,39.590717,43.481434,40.977413
2015-01-05,41.062634,37.980705,34.788544,29.756659,29.433289,34.755787,44.538898,55.089134,57.169266,57.658607,...,52.617409,51.011978,52.351734,62.277363,70.678871,65.034851,57.953537,50.226311,51.386963,49.472221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-12-27,50.551834,46.444595,42.371906,36.194542,34.270054,38.244186,46.993752,55.843174,58.677094,61.153885,...,58.760925,56.520859,58.666046,69.170273,77.202713,70.952469,63.990898,56.794037,57.636528,55.633186
2016-12-28,48.304089,45.168564,42.937218,38.274811,37.419586,41.560776,49.965996,59.251595,62.384789,62.366318,...,57.130566,56.582893,57.956825,66.709396,73.946754,68.98587,60.69775,54.918777,55.02158,51.425888
2016-12-29,49.327412,46.058777,43.709457,39.369034,37.774952,41.055817,49.926762,59.212761,62.323597,63.529427,...,56.083359,55.492271,57.067108,64.750023,72.666893,71.688507,63.947201,56.115387,56.907631,54.369781


In [18]:
forecast.to_csv("Resultados/dnn_FR_final.csv")

# DNN + Estandarización móvil

In [2]:
with open('dict_new_df_fr_epftoolbox_final.pkl', 'rb') as handle:
    dict_new_df = pickle.load(handle)

In [3]:
df_nuevo = pd.DataFrame(dict_new_df)

In [4]:
df_nuevo['Date'] = pd.to_datetime(df_nuevo.Date)

In [5]:
df_scalers = pd.DataFrame({'Date':dict_new_df['Date'], 'scaler':dict_new_df['scaler']})

In [6]:
df_scalers

,Date,scaler
0,2011-01-16 00:00:00,StandardScaler()
1,2011-01-16 01:00:00,StandardScaler()
2,2011-01-16 02:00:00,StandardScaler()
3,2011-01-16 03:00:00,StandardScaler()
4,2011-01-16 04:00:00,StandardScaler()
...,...,...
52243,2016-12-31 19:00:00,StandardScaler()
52244,2016-12-31 20:00:00,StandardScaler()
52245,2016-12-31 21:00:00,StandardScaler()
52246,2016-12-31 22:00:00,StandardScaler()


In [7]:
df_nuevo = df_nuevo[['Date', 'Price', 'Exogenous 1', 'Exogenous 2']]

In [8]:
df_nuevo = df_nuevo.set_index('Date')

In [9]:
df_nuevo.to_csv("Data/EPFToolbox/FR_dnn_estandarizacion_movil.csv")

In [10]:
from epftoolbox.models import hyperparameter_optimizer

In [11]:
# Number of layers in DNN
nlayers = 2

# Market under study. If it not one of the standard ones, the file name
# has to be provided, where the file has to be a csv file
dataset = "FR_dnn_estandarizacion_movil"

# Number of years (a year is 364 days) in the test dataset.
years_test = 1.5

# Optional parameters for selecting the test dataset, if either of them is not provided, 
# the test dataset is built using the years_test parameter. They should either be one of
# the date formats existing in python or a string with the following format
# "%d/%m/%Y %H:%M"
begin_test_date = "01/01/2015 00:00"
end_test_date = "31/12/2016 23:00"

# Boolean that selects whether the validation and training datasets are shuffled
shuffle_train = 1

# Boolean that selects whether a data augmentation technique for DNNs is used
data_augmentation = 0

# Boolean that selects whether we start a new hyperparameter optimization or we restart an existing one
new_hyperopt = 1

# Number of years used in the training dataset for recalibration
calibration_window = None

# Unique identifier to read the trials file of hyperparameter optimization
experiment_id = 2

# Number of iterations for hyperparameter optimization
max_evals = 100

path_datasets_folder = "./Data/EPFToolbox/"
path_hyperparameters_folder = "./experimental_files/"

# Check documentation of the hyperparameter_optimizer for each of the function parameters
# In this example, we optimize a model for the PJM market.
# We consider two directories, one for storing the datasets and the other one for the experimental files.
# We start a hyperparameter optimization from scratch. We employ 1500 iterations in hyperopt,
# 2 years of test data, a DNN with 2 hidden layers, a calibration window of 4 years,
# we avoid data augmentation,  and we provide an experiment_id equal to 1
hyperparameter_optimizer(path_datasets_folder=path_datasets_folder, 
                         path_hyperparameters_folder=path_hyperparameters_folder, 
                         new_hyperopt=new_hyperopt, max_evals=max_evals, nlayers=nlayers, dataset=dataset, 
                         years_test=years_test, calibration_window=calibration_window, 
                         shuffle_train=shuffle_train, data_augmentation=0, experiment_id=experiment_id,
                         begin_test_date=begin_test_date, end_test_date=end_test_date)

Test datasets: 2015-01-01 00:00:00 - 2016-12-31 23:00:00


Tested 1/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.5 | sMAPE: 77.07 %

Best MAE - Test Dataset
  MAE: 0.5 | sMAPE: 81.08 %


Tested 2/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.5 | sMAPE: 77.07 %

Best MAE - Test Dataset
  MAE: 0.5 | sMAPE: 81.08 %


Tested 3/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.4 | sMAPE: 73.99 %

Best MAE - Test Dataset
  MAE: 0.5 | sMAPE: 78.78 %


Tested 4/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.4 | sMAPE: 67.83 %

Best MAE - Test Dataset
  MAE: 0.4 | sMAPE: 73.32 %


Tested 5/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.4 | sMAPE: 67.83 %

Best MAE - Test Dataset
  MAE: 0.4 | sMAPE: 73.32 %


Tested 6/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.4 | sMAPE: 67.83 %

Best MAE - Test Dataset
  MAE: 0.4 | sMAPE: 73.32 %


Tested 7/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.4 | sMAPE: 67.83 %

Best MAE - Test Dataset
  MAE



Tested 60/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.3 | sMAPE: 59.66 %

Best MAE - Test Dataset
  MAE: 0.4 | sMAPE: 65.72 %


Tested 61/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.3 | sMAPE: 59.66 %

Best MAE - Test Dataset
  MAE: 0.4 | sMAPE: 65.72 %


Tested 62/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.3 | sMAPE: 59.66 %

Best MAE - Test Dataset
  MAE: 0.4 | sMAPE: 65.72 %


Tested 63/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.3 | sMAPE: 59.66 %

Best MAE - Test Dataset
  MAE: 0.4 | sMAPE: 65.72 %


Tested 64/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.3 | sMAPE: 59.66 %

Best MAE - Test Dataset
  MAE: 0.4 | sMAPE: 65.72 %


Tested 65/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.3 | sMAPE: 59.66 %

Best MAE - Test Dataset
  MAE: 0.4 | sMAPE: 65.72 %


Tested 66/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.3 | sMAPE: 59.66 %

Best MAE - Test Dataset
  MAE: 0.4 | sMAPE: 65.72 %


Tested 67/100 iterations.

In [13]:
"""
Example for using the DNN model for forecasting prices with daily recalibration
"""

# Author: Jesus Lago

# License: AGPL-3.0 License

import pandas as pd
import numpy as np
import argparse
import os

from epftoolbox.data import read_data
from epftoolbox.evaluation import MAE, sMAPE
from epftoolbox.models import DNN


path_datasets_folder = "./Data/EPFToolbox/"
path_hyperparameters_folder = "./experimental_files/"
path_recalibration_folder = os.path.join('.', 'Resultados')
path_hyperparameter_folder = os.path.join('.', 'experimental_files')

dataset = "FR_dnn_estandarizacion_movil"

# Number of years (a year is 364 days) in the test dataset.
years_test = 1.5

# Optional parameters for selecting the test dataset, if either of them is not provided, 
# the test dataset is built using the years_test parameter. They should either be one of
# the date formats existing in python or a string with the following format
# "%d/%m/%Y %H:%M"
begin_test_date = "01/01/2015 00:00"
end_test_date = "31/12/2016 23:00"

# Defining train and testing data
df_train, df_test = read_data(dataset=dataset, years_test=years_test, path=path_datasets_folder,
                              begin_test_date=begin_test_date, end_test_date=end_test_date)

# Defining unique name to save the forecast
forecast_file_name = 'dnn_FR_estandarizacion_movil_final.csv'

forecast_file_path = os.path.join(path_recalibration_folder, forecast_file_name)

# # Defining empty forecast array and the real values to be predicted in a more friendly format
forecast = pd.DataFrame(index=df_test.index[::24], columns=['h' + str(k) for k in range(24)])
real_values = df_test.loc[:, ['Price']].values.reshape(-1, 24)
real_values = pd.DataFrame(real_values, index=forecast.index, columns=forecast.columns)

# # If we are not starting a new recalibration but re-starting an old one, we import the
# # existing files and print metrics 
# if not new_recalibration:
#     # Import existinf forecasting file
#     forecast = pd.read_csv(forecast_file_path, index_col=0)
#     forecast.index = pd.to_datetime(forecast.index)

#     # Reading dates to still be forecasted by checking NaN values
#     forecast_dates = forecast[forecast.isna().any(axis=1)].index

#     # If all the dates to be forecasted have already been forecast, we print information
#     # and exit the script
#     if len(forecast_dates) == 0:

#         mae = np.mean(MAE(forecast.values.squeeze(), real_values.values))
#         smape = np.mean(sMAPE(forecast.values.squeeze(), real_values.values)) * 100
#         print('{} - sMAPE: {:.2f}%  |  MAE: {:.3f}'.format('Final metrics', smape, mae))
    
# else:
#     forecast_dates = forecast.index

forecast_dates = forecast.index

nlayers = 2
shuffle_train = 1
data_augmentation = 0
calibration_window = None
experiment_id = 2

model = DNN(
    experiment_id=experiment_id, path_hyperparameter_folder=path_hyperparameter_folder, nlayers=nlayers, 
    dataset=dataset, years_test=years_test, shuffle_train=shuffle_train, data_augmentation=data_augmentation,
    calibration_window=calibration_window)


# For loop over the recalibration dates
for date in forecast_dates:

    # For simulation purposes, we assume that the available data is
    # the data up to current date where the prices of current date are not known
    data_available = pd.concat([df_train, df_test.loc[:date + pd.Timedelta(hours=23), :]], axis=0)

    # We extract real prices for current date and set them to NaN in the dataframe of available data
    data_available.loc[date:date + pd.Timedelta(hours=23), 'Price'] = np.NaN

    # Recalibrating the model with the most up-to-date available data and making a prediction
    # for the next day
    Yp = model.recalibrate_and_forecast_next_day(df=data_available, next_day_date=date)

    # Saving the current prediction
    forecast.loc[date, :] = Yp

    # Computing metrics up-to-current-date
    mae = np.mean(MAE(forecast.loc[:date].values.squeeze(), real_values.loc[:date].values)) 
    smape = np.mean(sMAPE(forecast.loc[:date].values.squeeze(), real_values.loc[:date].values)) * 100

    # Pringint information
    print('{} - sMAPE: {:.2f}%  |  MAE: {:.3f}'.format(str(date)[:10], smape, mae))

#     # Saving forecast
#     forecast.to_csv(forecast_file_path)

Test datasets: 2015-01-01 00:00:00 - 2016-12-31 23:00:00
2015-01-01 - sMAPE: 79.80%  |  MAE: 0.313
2015-01-02 - sMAPE: 67.83%  |  MAE: 0.279
2015-01-03 - sMAPE: 84.45%  |  MAE: 0.315
2015-01-04 - sMAPE: 79.62%  |  MAE: 0.334
2015-01-05 - sMAPE: 75.06%  |  MAE: 0.363
2015-01-06 - sMAPE: 71.86%  |  MAE: 0.350
2015-01-07 - sMAPE: 68.19%  |  MAE: 0.333
2015-01-08 - sMAPE: 67.46%  |  MAE: 0.326
2015-01-09 - sMAPE: 72.08%  |  MAE: 0.368
2015-01-10 - sMAPE: 70.21%  |  MAE: 0.436
2015-01-11 - sMAPE: 68.50%  |  MAE: 0.440
2015-01-12 - sMAPE: 66.70%  |  MAE: 0.414
2015-01-13 - sMAPE: 65.87%  |  MAE: 0.399
2015-01-14 - sMAPE: 67.29%  |  MAE: 0.401
2015-01-15 - sMAPE: 66.09%  |  MAE: 0.390
2015-01-16 - sMAPE: 67.93%  |  MAE: 0.409
2015-01-17 - sMAPE: 68.15%  |  MAE: 0.398
2015-01-18 - sMAPE: 68.98%  |  MAE: 0.393
2015-01-19 - sMAPE: 66.78%  |  MAE: 0.387
2015-01-20 - sMAPE: 65.22%  |  MAE: 0.379
2015-01-21 - sMAPE: 65.35%  |  MAE: 0.380
2015-01-22 - sMAPE: 64.40%  |  MAE: 0.379
2015-01-23 - sMAPE:

2015-07-14 - sMAPE: 68.36%  |  MAE: 0.382
2015-07-15 - sMAPE: 68.25%  |  MAE: 0.382
2015-07-16 - sMAPE: 68.11%  |  MAE: 0.382
2015-07-17 - sMAPE: 68.11%  |  MAE: 0.382
2015-07-18 - sMAPE: 68.21%  |  MAE: 0.381
2015-07-19 - sMAPE: 68.24%  |  MAE: 0.381
2015-07-20 - sMAPE: 68.06%  |  MAE: 0.381
2015-07-21 - sMAPE: 68.01%  |  MAE: 0.382
2015-07-22 - sMAPE: 67.94%  |  MAE: 0.382
2015-07-23 - sMAPE: 67.79%  |  MAE: 0.381
2015-07-24 - sMAPE: 67.72%  |  MAE: 0.381
2015-07-25 - sMAPE: 67.82%  |  MAE: 0.385
2015-07-26 - sMAPE: 67.61%  |  MAE: 0.385
2015-07-27 - sMAPE: 67.99%  |  MAE: 0.388
2015-07-28 - sMAPE: 68.35%  |  MAE: 0.388
2015-07-29 - sMAPE: 68.54%  |  MAE: 0.388
2015-07-30 - sMAPE: 68.72%  |  MAE: 0.388
2015-07-31 - sMAPE: 68.56%  |  MAE: 0.387
2015-08-01 - sMAPE: 68.96%  |  MAE: 0.388
2015-08-02 - sMAPE: 69.05%  |  MAE: 0.389
2015-08-03 - sMAPE: 69.04%  |  MAE: 0.389
2015-08-04 - sMAPE: 69.08%  |  MAE: 0.390
2015-08-05 - sMAPE: 68.98%  |  MAE: 0.390
2015-08-06 - sMAPE: 68.85%  |  MAE

2016-01-26 - sMAPE: 66.81%  |  MAE: 0.380
2016-01-27 - sMAPE: 66.75%  |  MAE: 0.380
2016-01-28 - sMAPE: 66.74%  |  MAE: 0.379
2016-01-29 - sMAPE: 66.77%  |  MAE: 0.379
2016-01-30 - sMAPE: 66.71%  |  MAE: 0.379
2016-01-31 - sMAPE: 66.63%  |  MAE: 0.379
2016-02-01 - sMAPE: 66.62%  |  MAE: 0.379
2016-02-02 - sMAPE: 66.58%  |  MAE: 0.379
2016-02-03 - sMAPE: 66.62%  |  MAE: 0.379
2016-02-04 - sMAPE: 66.59%  |  MAE: 0.378
2016-02-05 - sMAPE: 66.63%  |  MAE: 0.378
2016-02-06 - sMAPE: 66.65%  |  MAE: 0.378
2016-02-07 - sMAPE: 66.60%  |  MAE: 0.379
2016-02-08 - sMAPE: 66.64%  |  MAE: 0.378
2016-02-09 - sMAPE: 66.59%  |  MAE: 0.378
2016-02-10 - sMAPE: 66.52%  |  MAE: 0.378
2016-02-11 - sMAPE: 66.45%  |  MAE: 0.377
2016-02-12 - sMAPE: 66.43%  |  MAE: 0.377
2016-02-13 - sMAPE: 66.52%  |  MAE: 0.377
2016-02-14 - sMAPE: 66.44%  |  MAE: 0.376
2016-02-15 - sMAPE: 66.44%  |  MAE: 0.376
2016-02-16 - sMAPE: 66.47%  |  MAE: 0.377
2016-02-17 - sMAPE: 66.39%  |  MAE: 0.377
2016-02-18 - sMAPE: 66.34%  |  MAE

2016-08-09 - sMAPE: 65.54%  |  MAE: 0.364
2016-08-10 - sMAPE: 65.53%  |  MAE: 0.363
2016-08-11 - sMAPE: 65.54%  |  MAE: 0.363
2016-08-12 - sMAPE: 65.53%  |  MAE: 0.363
2016-08-13 - sMAPE: 65.53%  |  MAE: 0.363
2016-08-14 - sMAPE: 65.54%  |  MAE: 0.363
2016-08-15 - sMAPE: 65.57%  |  MAE: 0.363
2016-08-16 - sMAPE: 65.52%  |  MAE: 0.363
2016-08-17 - sMAPE: 65.48%  |  MAE: 0.363
2016-08-18 - sMAPE: 65.42%  |  MAE: 0.363
2016-08-19 - sMAPE: 65.39%  |  MAE: 0.363
2016-08-20 - sMAPE: 65.37%  |  MAE: 0.363
2016-08-21 - sMAPE: 65.38%  |  MAE: 0.364
2016-08-22 - sMAPE: 65.32%  |  MAE: 0.363
2016-08-23 - sMAPE: 65.24%  |  MAE: 0.363
2016-08-24 - sMAPE: 65.19%  |  MAE: 0.363
2016-08-25 - sMAPE: 65.15%  |  MAE: 0.363
2016-08-26 - sMAPE: 65.12%  |  MAE: 0.363
2016-08-27 - sMAPE: 65.20%  |  MAE: 0.363
2016-08-28 - sMAPE: 65.18%  |  MAE: 0.363
2016-08-29 - sMAPE: 65.20%  |  MAE: 0.363
2016-08-30 - sMAPE: 65.17%  |  MAE: 0.363
2016-08-31 - sMAPE: 65.12%  |  MAE: 0.363
2016-09-01 - sMAPE: 65.08%  |  MAE

In [14]:
forecast

,h0,h1,h2,h3,h4,h5,h6,h7,h8,h9,...,h14,h15,h16,h17,h18,h19,h20,h21,h22,h23
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-01,0.679311,0.317702,-0.007746,-0.409563,-0.652361,-0.59541,-0.698651,-0.827979,-0.377593,0.173806,...,-0.123357,-0.354017,-0.228002,0.375615,1.123857,1.308499,0.928671,0.654994,0.854893,0.783545
2015-01-02,0.10391,-0.173948,-0.441023,-0.808026,-0.919788,-0.695696,-0.577923,-0.115445,0.146776,0.347853,...,0.120198,-0.041122,0.021172,0.45624,1.102305,0.985893,0.578449,0.3601,0.647879,0.509617
2015-01-03,-0.127398,-0.323326,-0.638103,-1.126966,-1.316215,-1.155597,-0.993052,-0.680029,-0.411287,-0.112341,...,-0.355489,-0.486896,-0.500269,-0.077063,0.418519,0.425572,0.027972,-0.225128,0.119841,-0.00698
2015-01-04,-0.625545,-0.931548,-1.28271,-1.694419,-1.852168,-1.822497,-1.754178,-1.771223,-1.379182,-0.872931,...,-0.988158,-1.125751,-1.011864,-0.434681,0.304767,0.565636,0.336021,-0.031002,0.004139,-0.168619
2015-01-05,-0.463267,-0.743277,-1.179797,-1.549478,-1.557973,-0.833212,0.201272,1.023058,1.28345,1.281758,...,0.899753,0.832215,0.976131,1.934947,2.791052,2.445122,1.59374,0.702371,0.848681,0.547744
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-12-27,-0.455787,-0.658931,-0.902462,-1.282551,-1.399836,-0.987896,-0.396532,0.250287,0.422494,0.469239,...,-0.006835,-0.091097,-0.003386,0.647526,1.203612,0.916598,0.363832,-0.089953,0.046407,-0.124255
2016-12-28,-0.402027,-0.496614,-0.616232,-0.788908,-0.881188,-0.631824,-0.298415,0.347827,0.518236,0.409412,...,-0.11572,-0.084049,-0.01288,0.627375,1.185257,0.881169,0.287823,-0.092977,-0.086258,-0.237527
2016-12-29,-0.249004,-0.45752,-0.708146,-1.014029,-1.029189,-0.761253,-0.245611,0.417863,0.686191,0.790113,...,0.544219,0.47937,0.674603,1.319369,1.837968,1.456689,0.969636,0.54638,0.629177,0.335602


In [15]:
forecast.to_csv("Resultados/dnn_FR_estandarizacion_movil_final.csv")

In [16]:
df_test_original = pd.DataFrame(forecast.values.reshape((-1,)), columns=['Price'])
df_test_original['Date'] = df_scalers.tail(len(df_test_original)).Date.to_numpy()

In [17]:
predicciones = np.array([])
for cont in range(0, len(df_test_original)):
    
    print(df_test_original.iloc[cont].Date)
    
    # Acumulamos las predicciones
    scaler_obj = df_scalers[df_scalers.Date == df_test_original.iloc[cont].Date].scaler.values[0]

    predicciones = np.append(predicciones, scaler_obj.inverse_transform(np.array(df_test_original.iloc[cont].Price).reshape(1, -1))[0][0])

2015-01-01 00:00:00
2015-01-01 01:00:00
2015-01-01 02:00:00
2015-01-01 03:00:00
2015-01-01 04:00:00
2015-01-01 05:00:00
2015-01-01 06:00:00
2015-01-01 07:00:00
2015-01-01 08:00:00
2015-01-01 09:00:00
2015-01-01 10:00:00
2015-01-01 11:00:00
2015-01-01 12:00:00
2015-01-01 13:00:00
2015-01-01 14:00:00
2015-01-01 15:00:00
2015-01-01 16:00:00
2015-01-01 17:00:00
2015-01-01 18:00:00
2015-01-01 19:00:00
2015-01-01 20:00:00
2015-01-01 21:00:00
2015-01-01 22:00:00
2015-01-01 23:00:00
2015-01-02 00:00:00
2015-01-02 01:00:00
2015-01-02 02:00:00
2015-01-02 03:00:00
2015-01-02 04:00:00
2015-01-02 05:00:00
2015-01-02 06:00:00
2015-01-02 07:00:00
2015-01-02 08:00:00
2015-01-02 09:00:00
2015-01-02 10:00:00
2015-01-02 11:00:00
2015-01-02 12:00:00
2015-01-02 13:00:00
2015-01-02 14:00:00
2015-01-02 15:00:00
2015-01-02 16:00:00
2015-01-02 17:00:00
2015-01-02 18:00:00
2015-01-02 19:00:00
2015-01-02 20:00:00
2015-01-02 21:00:00
2015-01-02 22:00:00
2015-01-02 23:00:00
2015-01-03 00:00:00
2015-01-03 01:00:00


2015-01-20 22:00:00
2015-01-20 23:00:00
2015-01-21 00:00:00
2015-01-21 01:00:00
2015-01-21 02:00:00
2015-01-21 03:00:00
2015-01-21 04:00:00
2015-01-21 05:00:00
2015-01-21 06:00:00
2015-01-21 07:00:00
2015-01-21 08:00:00
2015-01-21 09:00:00
2015-01-21 10:00:00
2015-01-21 11:00:00
2015-01-21 12:00:00
2015-01-21 13:00:00
2015-01-21 14:00:00
2015-01-21 15:00:00
2015-01-21 16:00:00
2015-01-21 17:00:00
2015-01-21 18:00:00
2015-01-21 19:00:00
2015-01-21 20:00:00
2015-01-21 21:00:00
2015-01-21 22:00:00
2015-01-21 23:00:00
2015-01-22 00:00:00
2015-01-22 01:00:00
2015-01-22 02:00:00
2015-01-22 03:00:00
2015-01-22 04:00:00
2015-01-22 05:00:00
2015-01-22 06:00:00
2015-01-22 07:00:00
2015-01-22 08:00:00
2015-01-22 09:00:00
2015-01-22 10:00:00
2015-01-22 11:00:00
2015-01-22 12:00:00
2015-01-22 13:00:00
2015-01-22 14:00:00
2015-01-22 15:00:00
2015-01-22 16:00:00
2015-01-22 17:00:00
2015-01-22 18:00:00
2015-01-22 19:00:00
2015-01-22 20:00:00
2015-01-22 21:00:00
2015-01-22 22:00:00
2015-01-22 23:00:00


2015-02-08 16:00:00
2015-02-08 17:00:00
2015-02-08 18:00:00
2015-02-08 19:00:00
2015-02-08 20:00:00
2015-02-08 21:00:00
2015-02-08 22:00:00
2015-02-08 23:00:00
2015-02-09 00:00:00
2015-02-09 01:00:00
2015-02-09 02:00:00
2015-02-09 03:00:00
2015-02-09 04:00:00
2015-02-09 05:00:00
2015-02-09 06:00:00
2015-02-09 07:00:00
2015-02-09 08:00:00
2015-02-09 09:00:00
2015-02-09 10:00:00
2015-02-09 11:00:00
2015-02-09 12:00:00
2015-02-09 13:00:00
2015-02-09 14:00:00
2015-02-09 15:00:00
2015-02-09 16:00:00
2015-02-09 17:00:00
2015-02-09 18:00:00
2015-02-09 19:00:00
2015-02-09 20:00:00
2015-02-09 21:00:00
2015-02-09 22:00:00
2015-02-09 23:00:00
2015-02-10 00:00:00
2015-02-10 01:00:00
2015-02-10 02:00:00
2015-02-10 03:00:00
2015-02-10 04:00:00
2015-02-10 05:00:00
2015-02-10 06:00:00
2015-02-10 07:00:00
2015-02-10 08:00:00
2015-02-10 09:00:00
2015-02-10 10:00:00
2015-02-10 11:00:00
2015-02-10 12:00:00
2015-02-10 13:00:00
2015-02-10 14:00:00
2015-02-10 15:00:00
2015-02-10 16:00:00
2015-02-10 17:00:00


2015-02-27 11:00:00
2015-02-27 12:00:00
2015-02-27 13:00:00
2015-02-27 14:00:00
2015-02-27 15:00:00
2015-02-27 16:00:00
2015-02-27 17:00:00
2015-02-27 18:00:00
2015-02-27 19:00:00
2015-02-27 20:00:00
2015-02-27 21:00:00
2015-02-27 22:00:00
2015-02-27 23:00:00
2015-02-28 00:00:00
2015-02-28 01:00:00
2015-02-28 02:00:00
2015-02-28 03:00:00
2015-02-28 04:00:00
2015-02-28 05:00:00
2015-02-28 06:00:00
2015-02-28 07:00:00
2015-02-28 08:00:00
2015-02-28 09:00:00
2015-02-28 10:00:00
2015-02-28 11:00:00
2015-02-28 12:00:00
2015-02-28 13:00:00
2015-02-28 14:00:00
2015-02-28 15:00:00
2015-02-28 16:00:00
2015-02-28 17:00:00
2015-02-28 18:00:00
2015-02-28 19:00:00
2015-02-28 20:00:00
2015-02-28 21:00:00
2015-02-28 22:00:00
2015-02-28 23:00:00
2015-03-01 00:00:00
2015-03-01 01:00:00
2015-03-01 02:00:00
2015-03-01 03:00:00
2015-03-01 04:00:00
2015-03-01 05:00:00
2015-03-01 06:00:00
2015-03-01 07:00:00
2015-03-01 08:00:00
2015-03-01 09:00:00
2015-03-01 10:00:00
2015-03-01 11:00:00
2015-03-01 12:00:00


2015-03-18 23:00:00
2015-03-19 00:00:00
2015-03-19 01:00:00
2015-03-19 02:00:00
2015-03-19 03:00:00
2015-03-19 04:00:00
2015-03-19 05:00:00
2015-03-19 06:00:00
2015-03-19 07:00:00
2015-03-19 08:00:00
2015-03-19 09:00:00
2015-03-19 10:00:00
2015-03-19 11:00:00
2015-03-19 12:00:00
2015-03-19 13:00:00
2015-03-19 14:00:00
2015-03-19 15:00:00
2015-03-19 16:00:00
2015-03-19 17:00:00
2015-03-19 18:00:00
2015-03-19 19:00:00
2015-03-19 20:00:00
2015-03-19 21:00:00
2015-03-19 22:00:00
2015-03-19 23:00:00
2015-03-20 00:00:00
2015-03-20 01:00:00
2015-03-20 02:00:00
2015-03-20 03:00:00
2015-03-20 04:00:00
2015-03-20 05:00:00
2015-03-20 06:00:00
2015-03-20 07:00:00
2015-03-20 08:00:00
2015-03-20 09:00:00
2015-03-20 10:00:00
2015-03-20 11:00:00
2015-03-20 12:00:00
2015-03-20 13:00:00
2015-03-20 14:00:00
2015-03-20 15:00:00
2015-03-20 16:00:00
2015-03-20 17:00:00
2015-03-20 18:00:00
2015-03-20 19:00:00
2015-03-20 20:00:00
2015-03-20 21:00:00
2015-03-20 22:00:00
2015-03-20 23:00:00
2015-03-21 00:00:00


...

2016-10-05 09:00:00
2016-10-05 10:00:00
2016-10-05 11:00:00
2016-10-05 12:00:00
2016-10-05 13:00:00
2016-10-05 14:00:00
2016-10-05 15:00:00
2016-10-05 16:00:00
2016-10-05 17:00:00
2016-10-05 18:00:00
2016-10-05 19:00:00
2016-10-05 20:00:00
2016-10-05 21:00:00
2016-10-05 22:00:00
2016-10-05 23:00:00
2016-10-06 00:00:00
2016-10-06 01:00:00
2016-10-06 02:00:00
2016-10-06 03:00:00
2016-10-06 04:00:00
2016-10-06 05:00:00
2016-10-06 06:00:00
2016-10-06 07:00:00
2016-10-06 08:00:00
2016-10-06 09:00:00
2016-10-06 10:00:00
2016-10-06 11:00:00
2016-10-06 12:00:00
2016-10-06 13:00:00
2016-10-06 14:00:00
2016-10-06 15:00:00
2016-10-06 16:00:00
2016-10-06 17:00:00
2016-10-06 18:00:00
2016-10-06 19:00:00
2016-10-06 20:00:00
2016-10-06 21:00:00
2016-10-06 22:00:00
2016-10-06 23:00:00
2016-10-07 00:00:00
2016-10-07 01:00:00
2016-10-07 02:00:00
2016-10-07 03:00:00
2016-10-07 04:00:00
2016-10-07 05:00:00
2016-10-07 06:00:00
2016-10-07 07:00:00
2016-10-07 08:00:00
2016-10-07 09:00:00
2016-10-07 10:00:00


2016-10-24 06:00:00
2016-10-24 07:00:00
2016-10-24 08:00:00
2016-10-24 09:00:00
2016-10-24 10:00:00
2016-10-24 11:00:00
2016-10-24 12:00:00
2016-10-24 13:00:00
2016-10-24 14:00:00
2016-10-24 15:00:00
2016-10-24 16:00:00
2016-10-24 17:00:00
2016-10-24 18:00:00
2016-10-24 19:00:00
2016-10-24 20:00:00
2016-10-24 21:00:00
2016-10-24 22:00:00
2016-10-24 23:00:00
2016-10-25 00:00:00
2016-10-25 01:00:00
2016-10-25 02:00:00
2016-10-25 03:00:00
2016-10-25 04:00:00
2016-10-25 05:00:00
2016-10-25 06:00:00
2016-10-25 07:00:00
2016-10-25 08:00:00
2016-10-25 09:00:00
2016-10-25 10:00:00
2016-10-25 11:00:00
2016-10-25 12:00:00
2016-10-25 13:00:00
2016-10-25 14:00:00
2016-10-25 15:00:00
2016-10-25 16:00:00
2016-10-25 17:00:00
2016-10-25 18:00:00
2016-10-25 19:00:00
2016-10-25 20:00:00
2016-10-25 21:00:00
2016-10-25 22:00:00
2016-10-25 23:00:00
2016-10-26 00:00:00
2016-10-26 01:00:00
2016-10-26 02:00:00
2016-10-26 03:00:00
2016-10-26 04:00:00
2016-10-26 05:00:00
2016-10-26 06:00:00
2016-10-26 07:00:00


2016-11-12 07:00:00
2016-11-12 08:00:00
2016-11-12 09:00:00
2016-11-12 10:00:00
2016-11-12 11:00:00
2016-11-12 12:00:00
2016-11-12 13:00:00
2016-11-12 14:00:00
2016-11-12 15:00:00
2016-11-12 16:00:00
2016-11-12 17:00:00
2016-11-12 18:00:00
2016-11-12 19:00:00
2016-11-12 20:00:00
2016-11-12 21:00:00
2016-11-12 22:00:00
2016-11-12 23:00:00
2016-11-13 00:00:00
2016-11-13 01:00:00
2016-11-13 02:00:00
2016-11-13 03:00:00
2016-11-13 04:00:00
2016-11-13 05:00:00
2016-11-13 06:00:00
2016-11-13 07:00:00
2016-11-13 08:00:00
2016-11-13 09:00:00
2016-11-13 10:00:00
2016-11-13 11:00:00
2016-11-13 12:00:00
2016-11-13 13:00:00
2016-11-13 14:00:00
2016-11-13 15:00:00
2016-11-13 16:00:00
2016-11-13 17:00:00
2016-11-13 18:00:00
2016-11-13 19:00:00
2016-11-13 20:00:00
2016-11-13 21:00:00
2016-11-13 22:00:00
2016-11-13 23:00:00
2016-11-14 00:00:00
2016-11-14 01:00:00
2016-11-14 02:00:00
2016-11-14 03:00:00
2016-11-14 04:00:00
2016-11-14 05:00:00
2016-11-14 06:00:00
2016-11-14 07:00:00
2016-11-14 08:00:00


2016-12-01 06:00:00
2016-12-01 07:00:00
2016-12-01 08:00:00
2016-12-01 09:00:00
2016-12-01 10:00:00
2016-12-01 11:00:00
2016-12-01 12:00:00
2016-12-01 13:00:00
2016-12-01 14:00:00
2016-12-01 15:00:00
2016-12-01 16:00:00
2016-12-01 17:00:00
2016-12-01 18:00:00
2016-12-01 19:00:00
2016-12-01 20:00:00
2016-12-01 21:00:00
2016-12-01 22:00:00
2016-12-01 23:00:00
2016-12-02 00:00:00
2016-12-02 01:00:00
2016-12-02 02:00:00
2016-12-02 03:00:00
2016-12-02 04:00:00
2016-12-02 05:00:00
2016-12-02 06:00:00
2016-12-02 07:00:00
2016-12-02 08:00:00
2016-12-02 09:00:00
2016-12-02 10:00:00
2016-12-02 11:00:00
2016-12-02 12:00:00
2016-12-02 13:00:00
2016-12-02 14:00:00
2016-12-02 15:00:00
2016-12-02 16:00:00
2016-12-02 17:00:00
2016-12-02 18:00:00
2016-12-02 19:00:00
2016-12-02 20:00:00
2016-12-02 21:00:00
2016-12-02 22:00:00
2016-12-02 23:00:00
2016-12-03 00:00:00
2016-12-03 01:00:00
2016-12-03 02:00:00
2016-12-03 03:00:00
2016-12-03 04:00:00
2016-12-03 05:00:00
2016-12-03 06:00:00
2016-12-03 07:00:00


2016-12-19 23:00:00
2016-12-20 00:00:00
2016-12-20 01:00:00
2016-12-20 02:00:00
2016-12-20 03:00:00
2016-12-20 04:00:00
2016-12-20 05:00:00
2016-12-20 06:00:00
2016-12-20 07:00:00
2016-12-20 08:00:00
2016-12-20 09:00:00
2016-12-20 10:00:00
2016-12-20 11:00:00
2016-12-20 12:00:00
2016-12-20 13:00:00
2016-12-20 14:00:00
2016-12-20 15:00:00
2016-12-20 16:00:00
2016-12-20 17:00:00
2016-12-20 18:00:00
2016-12-20 19:00:00
2016-12-20 20:00:00
2016-12-20 21:00:00
2016-12-20 22:00:00
2016-12-20 23:00:00
2016-12-21 00:00:00
2016-12-21 01:00:00
2016-12-21 02:00:00
2016-12-21 03:00:00
2016-12-21 04:00:00
2016-12-21 05:00:00
2016-12-21 06:00:00
2016-12-21 07:00:00
2016-12-21 08:00:00
2016-12-21 09:00:00
2016-12-21 10:00:00
2016-12-21 11:00:00
2016-12-21 12:00:00
2016-12-21 13:00:00
2016-12-21 14:00:00
2016-12-21 15:00:00
2016-12-21 16:00:00
2016-12-21 17:00:00
2016-12-21 18:00:00
2016-12-21 19:00:00
2016-12-21 20:00:00
2016-12-21 21:00:00
2016-12-21 22:00:00
2016-12-21 23:00:00
2016-12-22 00:00:00


In [18]:
df_test_original['Price'] = predicciones

In [19]:
df_test_original = df_test_original.set_index(df_test_original.Date).drop('Date', axis = 1)

In [20]:
forecast_final = df_test_original.loc[:, ['Price']].values.reshape(-1, 24)
forecast_final = pd.DataFrame(forecast_final, index=df_test_original.index[::24], columns=['h' + str(k) for k in range(24)])

In [21]:
forecast_final

,h0,h1,h2,h3,h4,h5,h6,h7,h8,h9,...,h14,h15,h16,h17,h18,h19,h20,h21,h22,h23
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-01,45.947344,41.365191,37.241248,32.149597,29.072962,29.794623,28.486399,26.847604,32.554708,39.541800,...,35.776275,32.853458,34.450267,42.099037,51.580442,53.920145,49.107120,45.639203,48.172247,47.268156
2015-01-02,41.444716,38.439272,35.550462,31.580787,30.371921,32.795802,34.069689,39.072072,41.908375,44.083310,...,41.620887,39.875974,40.549779,45.255677,52.243808,50.984643,46.577543,44.215779,47.328527,45.833029
2015-01-03,39.684710,37.609716,34.276030,29.098676,27.094421,28.795462,30.516907,33.832015,36.678152,39.844168,...,37.269084,35.877411,35.735775,40.217785,45.466297,45.540995,41.330167,38.649684,42.303116,40.960001
2015-01-04,36.972739,34.131838,30.871684,27.049416,25.584886,25.860349,26.494617,26.336377,29.976050,34.676033,...,33.606280,32.328872,33.386191,38.744698,45.609662,48.031549,45.899825,42.492420,42.818666,41.214793
2015-01-05,38.845167,36.405158,32.601325,29.379922,29.305892,35.621470,44.635967,51.797015,54.066076,54.051336,...,50.722534,50.134013,51.388098,59.743230,67.203334,64.188894,56.769944,49.002549,50.277500,47.655129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-12-27,48.427655,45.494584,41.978403,36.490551,34.797153,40.744888,49.283194,58.622176,61.108558,61.783479,...,54.909767,53.693161,54.959571,64.357642,72.386598,68.242594,60.261572,53.709681,55.678495,53.214421
2016-12-28,47.985209,46.723317,45.127477,42.823789,41.592672,44.919460,49.367504,57.989086,60.262531,58.810699,...,51.804861,52.227389,53.176852,61.718569,69.161333,65.104455,57.188566,52.108271,52.197913,50.179821
2016-12-29,48.433949,45.862745,42.772288,39.000455,38.813515,42.117430,48.475790,56.657054,59.965805,61.247254,...,58.215148,57.415502,59.822905,67.773495,74.168312,69.466780,63.460950,58.241799,59.262764,55.642705


In [22]:
forecast_final.to_csv("Resultados/dnn_FR_estandarizacion_movil_final.csv")

In [23]:
_, df_test = read_data(dataset="FR_dnn", years_test=years_test, path=path_datasets_folder,
                              begin_test_date=begin_test_date, end_test_date=end_test_date)

real_values = df_test.loc[:, ['Price']].values.reshape(-1, 24)
real_values = pd.DataFrame(real_values, index=forecast.index, columns=forecast.columns)

Test datasets: 2015-01-01 00:00:00 - 2016-12-31 23:00:00


In [24]:
mae = np.mean(MAE(forecast_final.values.squeeze(), real_values.values)) 
smape = np.mean(sMAPE(forecast_final.values.squeeze(), real_values.values)) * 100
print('sMAPE: {:.2f}%  |  MAE: {:.3f}'.format(smape, mae))

sMAPE: 12.02%  |  MAE: 4.289
